<a href="https://colab.research.google.com/github/fxr1115/Learning/blob/main/Python3-Algorithm/array-string/1_one_dimensional_array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from typing import List

### 列表和数组

列表list：
- 可以存储不同类型的元素
- 适合动态操作，但在处理大量数值时，性能和内存效率差

数组array：
- `import array`
- 所有元素必须是相同类型
- 数值计算下更高效，使用的内存比list少
- 更常用的是`numpy.ndarray`



#### 数组的四种操作

读取元素：
- 访问索引来读取
- 对于数组，计算机会在内存中为其申请一段**连续**空间，并记下索引为0处的内存地址
- 复杂度是常数级别$O(1)$

查找元素
- 只保存了索引为0的内存地址，查找时从头逐步查
- 时间复杂度是$O(N)$，N是数组长度

插入元素
- 数组末尾插入只需一步
- 其他位置，需要为该元素要插入位置**腾出**空间（依次腾）
- 频繁插入会浪费时间，链表更好使

删除元素
- 数组会留下**空缺**位置，内存是连续，所以后面元素要对该位置进行**填补**操作（依次）

### 题目1 寻找中心下标
给一个整数数组`nums`，计算数组的中心下标（使得左侧的所有元素相加之和等于右侧所有元素之和）

In [ ]:
class Solution1:
    def pivotIndex(self, nums: List[int]) -> int:
        all_sum = sum(nums)
        for i in range(len(nums)):
            left_sum = 0
            right_sum = 0
            for j in range(i):
                left_sum += nums[j]
            right_sum = all_sum - left_sum - nums[i]
            if left_sum == right_sum:
                return i
                break
        return -1

In [5]:
Solution1().pivotIndex([1,7,3,6,5,6])

3

**代码改进**：（效率问题，冗余计算，变量名优化）


- 每次计算左侧和（left_sum）都需要*从头*到当前位置遍历一遍，导致时间复杂度为$O(n^2)$

- 循环中的 break 是多余的，因为在满足条件时直接返回即可
- left_sum 和 right_sum 的更新可以整合到循环中，无需额外定义并重复赋值

In [ ]:
class Solution2:
    def pivotIndex(self, nums: List[int]) -> int:
        all_sum = sum(nums)
        left_sum = 0
        for i in range(len(nums)):
            right_sum = all_sum - left_sum - nums[i]
            if right_sum == left_sum:
                return i
            left_sum += nums[i]
        return -1

In [7]:
Solution2().pivotIndex([1,7,3,6,5,6])

3

### 题目2 搜索插入位置
给定一个**排序**数组和一个目标值，在数组中找到目标值，并返回其索引；如果目标值不存在于数组中，返回它将会被按顺序插入的位置
- 请必须使用时间复杂度为$O(log n)$的算法。

**自己**：**错误**
- 使用的是线性查找，时间复杂度为O(n)
- 当`target`比所有元素都小的话无法正确插入位置

In [ ]:
class Solution:
    def searchInsert(self, nums: List[int], target: int) -> int:
        for i in range(len(nums)):
            if nums[i] == target:
                return i
            if nums[i] > target:
                return i
      return i + 1

**线性改进**

In [ ]:
class Solution:
    def searchInsert(self, nums: List[int], target: int) -> int:
        for i in range(len(nums)):
            if nums[i] >= target:
                return i
        return len(nums)

**$O(logn)$改进**：使用**二分法**

In [ ]:
class Solution3:
    def searchInsert(self, nums: List[int], target: int) -> int:
        left, right = 0, len(nums) - 1
        while left <= right:
            mid = (left + right) // 2
            if nums[mid] == target:
                return mid
            elif nums[mid] > target:
                right = mid
            elif nums[mid] < target:
                left = mid
        return left

In [10]:
Solution3().searchInsert([1,3,5,6], 5)

2

### 题目3 合并区间
以数组`intervals`表示若干个区间的集合，其中单个区间为 `intervals[i] = [starti, endi]`
- 合并所有重叠的区间，并返回一个不重叠的区间数组，该数组需恰好覆盖输入中的所有区间

**问题**：
- 如果多个重叠怎么办？
- 怎么将需要写入的区间写出来？

**尽力**版本

In [ ]:
class Solution:
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        output = []
        for i in range(len(intervals)):
            for j in range(i + 1, len(intervals)):
                output.append([intervals[i][0], intervals[j][1]])
                break
        return output

**代码问题**：（逻辑问题，未排序区间，输出逻辑错误，嵌套循环效率低下）
- `if intervals[i][1] > intervals[j][0]`的逻辑仅检查当前两个区间是否有交集，没有动态更新合并区间的上下界
- 在合并区间问题中，排序是必要前置条件
- 直接将找到的交集区间追加到`output`上，而不是持续更新区间的边界
- 嵌套循环的时间复杂度是$O(n^2)$；排序后一次遍历即可完成合并，时间复杂度是$O(nlogn)$

**改进思路**
- 先对区间按起点**升序排列**
- 逐一遍历合并区间
- 要动态的更新

**注**：
- sorted()：适用于*任何可迭代对象*，返回一个*新*列表，是*内置函数*
- list.sort()：*只作用于列表*，在*原*列表上排序，是*列表*的方法

In [ ]:
class Solution4:
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        if not intervals:
            return []
        # 排序
        intervals.sort(key=lambda x: x[0])
        merged = [intervals[0]]
        # 遍历剩余区间
        for i in range(1, len(intervals)):
            prev = merged[-1]
            curr = intervals[i]
            if curr[0] <= prev[1]:             
                prev[1] = max(prev[1], curr[1])
            else:
                merged.append(curr)
        return merged

In [18]:
Solution4().merge([[1,3],[2,6],[8,10],[15,18]])

[[1, 6], [8, 10], [15, 18]]

**更简洁一点**

In [ ]:
class Solution4:
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        interval.sort(key lambda x:)
        ans = []
        for x in intervals:
            if ans and x[0] <= ans[-1][1]:
                ans[-1][1] = max(x[1], ans[-1][1])
             else:
                ans.append(x)
        return ans